In [15]:
import requests
import base64
import tarfile
import tempfile

import arxiv

In [22]:
def _extract_arxiv_id(link: str) -> str:
    return link.split("/")[-1]

In [23]:
def process_arxiv_paper(paper_url):
    """Downloads and processes an arXiv paper, returning its TeX content and PDF content as base64.

    Args:
        paper_url: The URL of the arXiv paper to process.

    Returns:
        A tuple containing:
        - The TeX content of the paper as a string.
        - The PDF content of the paper as a base64-encoded string.
    """
    
    arxiv_client = arxiv.Client()

    arxiv_paper_obj = next(
        arxiv_client.results(arxiv.Search(id_list=[_extract_arxiv_id(paper_url)]))
    )

    with tempfile.TemporaryDirectory() as temp_dir:
        try:
            # Download the source and PDF files
            arxiv_paper_obj.download_source(dirpath=temp_dir, filename="paper.tar.gz")
            arxiv_paper_obj.download_pdf(dirpath=temp_dir, filename="paper.pdf")

            # Extract the TeX content from the tarball
            with tarfile.open(f"{temp_dir}/paper.tar.gz", "r:gz") as tar:
                for member in tar.getmembers():
                    if member.name.endswith(".tex"):
                        file = tar.extractfile(member)
                        tex_content = file.read().decode("utf-8")
                        break

            # Read the PDF content and convert to base64
            with open(f"{temp_dir}/paper.pdf", "rb") as file:
                pdf_content = base64.b64encode(file.read()).decode("utf-8")

            return tex_content, pdf_content

        except Exception as e:
            raise ValueError(f"Error processing arXiv paper: {e}") from e

In [24]:
# Example usage
paper_url = "https://arxiv.org/abs/2204.08387"

In [25]:
tex_content, pdf_content = process_arxiv_paper(paper_url)

In [27]:
pdf_content

'...'